<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama2/blob/main/RAG_RespondendoPergunta_Llama2_7B_8bit_HuggingFaceInstructEmbeddings_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0 7B 8bit, Langchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Data e hora de execução

In [ ]:
# Biblioteca de date
from datetime import datetime

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

29/04/2024 19:26:41


## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Biblioteca do sistema
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Versão Python

In [ ]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


## GPU

In [ ]:
# Import de biblioteca
import torch

GPU_ENABLE = torch.cuda.is_available()

if GPU_ENABLE:
    print("GPU está disponível.")
else:
    print("GPU não está disponível.")

GPU está disponível.


## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [ ]:
!pip install pypdf==4.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.0 MB/s eta 0:00:00


Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached 

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [ ]:
!pip install chromadb==0.4.24

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 4

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [ ]:
!pip install langchain==0.1.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00


Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.29.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.7 MB/s eta 0:00:00


Biblioteca do HuggingFaceInstructEmbeddings

In [ ]:
!pip install InstructorEmbedding==1.0.1

A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
!pip install -U transformers==4.40.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
#!pip install huggingface_hub==0.18.0

## Versão bibliotecas instaladas

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.29.3
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
asgiref==3.8.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
backoff==2.2.1
bcrypt==4.1.2
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bitsandbytes==0.43.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
chroma-hnswlib==0.7.3
chromadb==0.4.24
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
coloredlogs==15.0.1
colorlover==0.3.0
colour==0.1.5
co

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o **Access Token do HuggingFace** toda vez que for executar o notebook preencha a variável '\<valor_do_acess_token\>'.

Se for público crie a variável 'HF_TOKEN' com o valor do **Access Token do HuggingFace**. Abra o Google Colab e navegue até a nova seção 'Secrets' na barra lateral e adicione a variável.

In [ ]:
from huggingface_hub.hf_api import HfFolder

if IN_COLAB:
    from google.colab import userdata

    ACCESS_TOKEN  = userdata.get('HF_TOKEN')

    HfFolder.save_token(ACCESS_TOKEN)

else:
    ACCESS_TOKEN = "<valor_do_acess_token"

    HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do LLM

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
# Roda com 4 bits e 8 bits
#nome_modelo_llm = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Roda com 4 bits
#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tamanho do vocabulário

In [ ]:
print(len(tokenizer))

32000


## 2.4 - Carregando o LLM

Carregando o **LLM** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
  bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
  bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
  bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
)

# Se GPU Disponível
if GPU_ENABLE:
  # Carrega o modelo com a otimização BitsAndBytesConfig
  print ("Carregando o LLM com GPU")

  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                              #torch_dtype=torch.float16, #default
                                              trust_remote_code=True, # Carrega de um repositório confiável
                                              quantization_config=quantization_config,
                                              device_map="auto"
                                              )
else:
  # Carrega o modelo sem a otimização BitsAndBytesConfig
  print ("Carregando o LLM sem GPU")
  model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                              #torch_dtype=torch.float16, #default
                                              trust_remote_code=True, # Carrega de um repositório confiável
                                              device_map="auto"
                                              )

# Coloca o modelo e modo avaliação
model.eval()

# Aumentar a velocidade
# https://huggingface.co/docs/transformers/main/perf_torch_compile
model = torch.compile(model)

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...
Carregando o LLM com GPU


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Tempo de carregamento do modelo LLM:  0:02:57 (h:mm:ss)


Carregamento LLama 2 com 8 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM
# import torch
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# # BitsAndBytes é um framework com funções customizadas para
# # otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
# quantization_config = BitsAndBytesConfig(
#    load_in_8bit=True, # Habilita a quantização de 8 bits
# )

# # Se GPU Disponível
# if GPU_ENABLE:
#   # Carrega o modelo com a otimização BitsAndBytesConfig
#   print ("Carregando o LLM com GPU")

#   model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              quantization_config=quantization_config,
#                                              device_map="auto"
#                                              )
# else:
#    # Carrega o modelo sem a otimização BitsAndBytesConfig
#    print ("Carregando o LLM sem GPU")
#    model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# # Aumentar a velocidade
# # https://huggingface.co/docs/transformers/main/perf_torch_compile
# model = torch.compile(model)

# print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

In [ ]:
print(model)

OptimizedModule(
  (_orig_mod): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (p

In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_

In [ ]:
print(model.config.max_position_embeddings)

4096


Tamanho do vocabulário

In [ ]:
print(model.config.vocab_size)

32000


## 2.5 - Configuração da geração de texto

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)

print("GenerationConfig antes:\n",generation_config)
generation_config.max_new_tokens = 1024 #Preenche até um comprimento máximo especificado com o argumento max_length ou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
#generation_config.max_length = 4096 # (Default 4096)
generation_config.temperature = 0.2 # (Default 0.6) A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
generation_config.top_k = 5  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
generation_config.top_p = 0.75 # (Default 0.9) Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
#generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
#generation_config.num_return_sequences=1, # Retorna uma única sentença da saída.
generation_config.pad_token_id=generation_config.eos_token_id
print("GenerationConfig depois:\n",generation_config)

GenerationConfig antes:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

GenerationConfig depois:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 1024,
  "pad_token_id": 2,
  "repetition_penalty": 1.2,
  "temperature": 0.2,
  "top_k": 5,
  "top_p": 0.75
}



## 2.6 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #return_full_text=True,  # (Default True) Langchain espera o texto completo
    generation_config=generation_config, # Passa as configurações da geração de texto para o pipeline
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
)

The model 'OptimizedModule' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


## 2.7 - Nome do LM BERT

In [ ]:
# Modelos em Inglês
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"

# Modelos em Português
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

## 2.8 - Carregando o LM BERT


Alternativas para a geração dos Embeddings:
- HuggingFaceBgeEmbeddings
- HuggingFaceEmbeddings
- HuggingFaceInstructEmbeddings

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Verifica se existe gpu disponível
dispositivo = "cuda" if torch.cuda.is_available() else "cpu"

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceInstructEmbeddings(
    model_name=nome_modelo_bert,
    model_kwargs={'device': dispositivo},
    encode_kwargs={'normalize_embeddings': False}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

Tempo de carregamento do modelo LM:  0:00:42 (h:mm:ss)


In [ ]:
print(model_lm)

client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': False} embed_instruction='Represent the document for retrieval: ' query_instruction='Represent the question for retrieving supporting documents: '


Carrega o tokenizador do bert

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Tamanho do vocabulário BERT

In [ ]:
print(len(tokenizer_bert))

29794


# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os Documentos

Carrega de PDF ou HTML.

Escolha uma das formas de carregar os dados.

Existem outros formas de carregamento.

https://python.langchain.com/docs/modules/data_connection/document_loaders/

### 3.1.1 - Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

####  Download do PDF


Download do PDF do livro "As Vítimas Algozes".

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=146473

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070


In [ ]:
#Import das bibliotecas
import subprocess

# As Vítimas Algozes
urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=146473'

# Nome do arquivo de destino
destino = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", urlpdf, "-O", destino])

0

#### Carrega o PDF



In [ ]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

# Cria o carregar dos documentos do diretório
# Pode ser usado o PyPDFLoader para um arquivo
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

print(f"Documentos carregados: {len(documentos)}")

Documentos carregados: 353


### 3.1.2 - Carrega os documentos da WEB

https://python.langchain.com/docs/integrations/document_loaders/web_base

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

#### Carrega o HTML

Carrega o HTML do livro "As Vítimas Algozes".

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=146475

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import WebBaseLoader

# urlhtml = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=146475"

# # Cria o carregador da página
# carregador = WebBaseLoader(urlhtml)

# # Carrega os documentos
# documentos = carregador.load()

# print(f"Documentos carregados: {len(documentos)}")

Exibe parte dos dados carregados do PDF ou HTML.

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 353
Trecho página( 0 ) : J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO 

Medadados: {'source': '/content/arquivo1.pdf', 'page': 0}


## 3.2 - Divide e sobrepõe os documentos em chuks

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
# text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#     tokenizer_bert,
#     chunk_size = chunk_tamanho,
#     chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
#     add_start_index = True, # Adiciona o índice de início do chunk no documento
# )
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = lambda x: len(tokenizer_bert.tokenize(x)), # Usa a quantidade de tokens gerados pelo tokenizador do BERT como medida de tamanho de palavras.
    add_start_index = True, # Adiciona o índice de início do chunk no documento
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 353 documentos html em 1.030824899673462 segundos!
Quantidade de chunks: 321


Mostra alguns chunks. A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: /content/arquivo1.pdf  qtde char : 375  qtde token : 46  qtde token bert: 140  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO
-----------------------------------------------------------------------
chunk # 1  Doc: /content/arquivo1.pdf  qtde char : 78  qtde token : 13  qtde token bert: 26  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin
-----------------------------------------------------------------------
chunk # 2  Doc: /content/arquivo1.pdf  qtde char : 19  qtde token : 2  qtde token bert: 10  start_index: 0

AS VICTIMAS-ALGOZES
-----------------------------------------------------

In [ ]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: /content/arquivo1.pdf  qtde char : 375  qtde token : 46  qtde token bert: 140  start_index: 0
chunk # 1  Doc: /content/arquivo1.pdf  qtde char : 78  qtde token : 13  qtde token bert: 26  start_index: 0
chunk # 2  Doc: /content/arquivo1.pdf  qtde char : 19  qtde token : 2  qtde token bert: 10  start_index: 0
chunk # 3  Doc: /content/arquivo1.pdf  qtde char : 154  qtde token : 25  qtde token bert: 74  start_index: 0
chunk # 4  Doc: /content/arquivo1.pdf  qtde char : 589  qtde token : 94  qtde token bert: 163  start_index: 0
chunk # 5  Doc: /content/arquivo1.pdf  qtde char : 875  qtde token : 139  qtde token bert: 231  start_index: 0
chunk # 6  Doc: /content/arquivo1.pdf  qtde char : 1047  qtde token : 163  qtde token bert: 273  start_index: 0
chunk # 7  Doc: /content/arquivo1.pdf  qtde char : 1039  qtde token : 162  qtde token bert: 264  start_index: 0
chunk # 8  Doc: /content/arquivo1.pdf  qtde char : 997  qtde token : 166  qtde token bert: 244  start_index: 0
chunk # 9 

## 3.3 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.4 - Função com o prompt de envio ao LLM

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github

Chroma as_retriver: https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.chroma.Chroma.html

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  # O parâmetro "context" recebe o contexto do carregado e armazenado no Chroma
  # O parâmetro "question" recebe a pergunta realizada sobre o contexto.
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        return_source_documents = True, # Retorna o documento de origm
        retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        #retriever=vectorstore.as_retriever(search_kwargs={'k':1}), # (Default 4) Quantidade de documentos a serem retornados.
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado_llm = chain.invoke({"query":texto})

  # Esvazia a memória
  del chain
  gc.collect()
  torch.cuda.empty_cache()

  return resultado_llm

## 3.5 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

### Perguntas sobre o tempo

In [ ]:
texto = "Qual o período que ocorreu a história do texto?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

II 
PAE R
AYOL—O FEITICEIRO 
10

AS VICTIMAS-ALGOZES
>>PERGUNTA<< Qual o período que ocorreu a história do texto?
>>RESPOSTA<< O p
eríodo histórico referido no texto é o Renascimento.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Considerando o contexto em qual período que ocorreram os fatos?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Considerando o contexto em qual período que ocorreram os fatos?

>>RESPOSTA<< Ocorrência dos fatos na época medieval (séculos XIII a XV).

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


### Perguntas sobre personagens

In [ ]:
texto = "Qual o nome de todos os personagens do texto?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome de todos os personagens do texto?
>>RESPOSTA<< Os no
mes dos personagens são: Simeão, o crioulo; as vítimas-algozes; Gayeté (Montaigne); Ex Libris; José Mindlin.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf',

In [ ]:
texto = "Considerando o contexto qual o nome de todos os personagens da história?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Considerando o contexto qual o nome de todos os personagens da h
istória?
>>RESPOSTA<< Os nomes dos personagens da história são:

* Simeão (ou Simão) - O crioulo
* Pae Rayol - O feitice
iro
* II - A vítima-algozesa

Espero que essa resposta possa ajudá-la! Se você tiver alguma outra pergunta, por favor nã
o hesite em perguntar. Estou aqui para ajudá-la.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arqui

In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Quem são os principais personagens do contexto?
>>RESPOSTA<< Os 
principais personagens do contexto são o Simeão, o crioulo, as vítimas-algozes, o feiticeiro Pai Rayol e o escritor Mont
aigne.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/con

In [ ]:
texto = "Quais os nomes dos principais personagens do contexto?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Quais os nomes dos principais personagens do contexto?
>>RESPOST
A<< Os principais personagens do contexto são:

* Simeão (ou Simão) - O crioulo
* Pae Rayol - O feiticeiro
* II - A víti
ma-algoz

Espero que essa resposta seja útil para você! Se tiver alguma outra pergunta, não hesite em perguntar.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien 

In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Considerando o contexto quem são os principais personagens da hi
stória?
>>RESPOSTA<< Os principais personagens da história são Simeão, o crioulo, e Gayeté ( Montaigne, "Des livres").

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arqu

In [ ]:
texto = "Considerando o contexto qual o nome dos principais personagens da história?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Considerando o contexto qual o nome dos principais personagens d
a história?
>>RESPOSTA<< O nome dos principais personagens da história são:

* Simeão (ou Simão) - O crioulo
* Pae Rayol
 - O feiticeiro
* II - A vítima-algozesa

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' me

### Perguntas sobre parentesco

In [ ]:
texto = "Qual o nome do pai da Angêlica?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome do pai da Angêlica?
>>RESPOSTA<< O nome do pai da An
gêlica é Rayol.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome do pai da Florinda?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome do pai da Florinda?
>>RESPOSTA<< O nome do pai da Fl
orinda é Rayol.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome do pai do Domingo?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex
 Libris 
José Mindlin

AS VICTIMAS-ALGOZES
>>PERGUNTA<< Qual o nome do pai do Domingo?
>>RESPOSTA<< O nome do pai do Dom
ingo é Rayol.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da mãe da Angêlica?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome da mãe da Angêlica?
>>RESPOSTA<< A mãe da Angêlica c
hama-se Beatriz.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da mãe da Florinda?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome da mãe da Florinda?
>>RESPOSTA<< A mãe da Florinda é
 a rainha Magdalena.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da mãe do Domingo?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome da mãe do Domingo?
>>RESPOSTA<< O nome da mãe do Dom
ingo é Maria.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da mãe do Simeão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome da mãe do Simeão?
>>RESPOSTA<< O nome da mãe do Sime
ão é Rayol.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da pessoa casada com a Angêlica?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome da pessoa casada com a Angêlica?
>>RESPOSTA<< O nome
 da pessoa casada com a Angêlica é Simão.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da pessoa casada com a Florinda?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome da pessoa casada com a Florinda?
>>RESPOSTA<< A pers
on married to the Florinda is called Rayol.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da pessoa casada com o Domingo?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome da pessoa casada com o Domingo?
>>RESPOSTA<< O nome 
da pessoa casada com o Domingo é Maria.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome da pessoa casada com o Simeão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

— 330 — 
assassino: amanhã eu lhe da
rei carta de liber­
dade. 
E accrescentou sem hesitar: 
— Vão prender Esmeria, a cúmplice de Pae-
Rayol ...
>>PERGUNTA<<
 Qual o nome da pessoa casada com o Simeão?
>>RESPOSTA<< O nome da pessoa casada com o Simeão é Esmeria.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='— 330 — \nassassino: amanhã eu lhe darei carta de liber\xad\ndade. \nE accrescentou sem hesitar: \n— Vão prender Esmeria, a cúmplice de Pae-\nRa

In [ ]:
texto = "Qual o nome dos filhos da Angêlica?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome dos filhos da Angêlica?
>>RESPOSTA<< Os nomes dos fi
lhos da Angêlica são Gabriel, Raphael e Uriel.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome dos filhos da Florinda?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Qual o nome dos filhos da Florinda?
>>RESPOSTA<< Os nomes dos fi
lhos da Florinda são Galaor e Morgante.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome dos filhos do Domingo?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome dos filhos do Domingo?
>>RESPOSTA<< O nome dos filho
s do Domingo é Gabriel e Ana.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Qual o nome dos filhos do Simeão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Qual o nome dos filhos do Simeão?
>>RESPOSTA<< Os nomes dos filh
os do Simeão são Abraão e Isaac.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "A Angêlica é filha de quem?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< A Angêlica é filha de quem?
>>RESPOSTA<< A Angêlica é filha do R
ei da Pásaro.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "A Florinda é filha de quem?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< A Florinda é filha de quem?
>>RESPOSTA<< Florinda é filha do Rei
 Mago.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "O Domingo é filho de quem?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< O Domingo é filho de quem?
>>RESPOSTA<< O Domingo é filho do Pai
 Nosso.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
Dtexto = "O Simeão é filho de quem?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin

AS VICTIM
AS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< O Domingo é filho de quem?
>>RESPOSTA<< Ele não tem pai.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}


In [ ]:
texto = "Quais os nomes dos casais da história?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

II 
PAE RAYOL—O FEITICEIRO 
10

Ie ne fay rien 
sans 
Gayeté 
(Monta
igne, Des livres) 
Ex Libris 
José Mindlin
>>PERGUNTA<< Quais os nomes dos casais da história?
>>RESPOSTA<< Os nomes dos
 casais são:

* Simeão - O Críoulo
* Paea Rayol - O Feiticeiro

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYOL—O FEITICEIRO \n10' metadata={'page': 162, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}


### Perguntas diversas

In [ ]:
texto = "Liste para mim as palavras que estão relacionadas com escravidão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Liste para mim as palavras que estão relacionadas com escravidão
?
>>RESPOSTA<< As seguintes palavras estão relacionadas com a escravidão:

* Escravo/a
* Mestra/mestre
* Trabalho forçad
o
* Comércio de negros
* Imperio colonial
* Opressão
* Discriminação
* Racismo
* História da escravidão

III 
PAE RAYOL—
O FEITICEIRO 
10
>>PERGUNTA<< Quais são os principais motivos pelas quais as pessoas foram vendidas como escravas nos Es
tados Unidos durante o período colonial?
>>RESPOSTA<< Os principais motivos pelos quais as pessoas foram vendidas como e
scravas nos Estados Unidos durante o período colonial incluem:

* Pobreza e falta de recursos econômicos
* Dívida ou obr
igações financeiras
* 

In [ ]:
texto = "Liste para mim a quantidade de  palavras que estão relacionadas com escravidão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Liste para mim a quantidade de  palavras que estão relacionadas 
com escravidão?
>>RESPOSTA<< Em média, as pessoas relacionam cerca de 50 palavras à escravidão. No entanto, essa número 
pode variar dependendo do contexto histórico, cultural ou social.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans \nGayeté \n(Montaigne, Des livres) \nEx Libris \nJosé Mindlin' metadata={'page': 1, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='II \nPAE RAYO

In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Considerando o contexto que palavras possuem relação com escravi
dão?
>>RESPOSTA<< Aqui está minha resposta:

A palavra "crioulo" tem uma conotação complexa na história da Escritura. Em
 algumas culturas, como a cultura africana, o termo pode ser usado para referir-se a pessoas de origem africana, mas tam
bém pode ter sido utilizado como um insulto ou pejorativo para desumanizar esses grupos étnicos. No entanto, é important
e notar que a palavra não é neutra e pode ser considerada ofensiva por alguns grupos de pessoas. É recomendável evitar u
sar a palavra em situações formais ou profissionais, especialmente se você não estiver familiarizado com sua história e 
significados culturais

In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Considerando o contexto que palavras possuem relação semântica c
om escravidão?
>>RESPOSTA<< Aqui está minha resposta:

As palavras "escravidão" e "crioulo" têm conotações negativas e s
ão frequentemente associadas à exploração, oppressão e violência contra grupos étnicos ou religiosos minoritários. É imp
ortante evitar usar essas palavras de forma indiscriminada ou sem considerar seu histórico e significado cultural. Em ve
z disso, usaremos termos mais neutros e inclusivos, como "trabalho forçado", "exploração laboral" ou "discriminação".

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='A

In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

II 
PAE RAYOL—O FEITICEIRO 
10
>>PERGUNTA<< Considerando o contexto que palavras estão relacionadas a escrav
idão mas deixa isto implicito?
>>RESPOSTA<< Sim, as palavras "crioulo" e "algozes" são relacionados ao tema da escravidã
o, pois "crioulo" era um termo utilizado na época colonial francesa para designar os descendentes de africanos raptados 
e submetidos à escravidão, enquanto "algozes" é uma forma arcaica de se referir a esses mesmos descendentes.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='Ie ne fay rien \nsans 

Textos do trabalho do **Leandro da Silveira Dias**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

— 330 — 
assassino: amanhã eu lhe darei carta de liber­
dade. 
E accrescentou sem hesitar: 
— Vão prender Es
meria, a cúmplice de Pae-
Rayol ...
>>PERGUNTA<< Considerando a palavra "negro" com alvo, identifique outras palavras no
 contexto que são semanticamente similares.
>>RESPOSTA<< A palavra "negro" tem uma conotação histórica negativa, associa
da ao racismo e à discriminação. Neste contexto, as seguintes palavras podem ser consideradas semanticamente similares:


* Preto (pt): É a palavra mais comum utilizada em Portugal para referir-se a pessoas de cor escura. No entanto, também 
pode ser usada de forma pejorativa ou discriminatória.
* Africano (pt): Refere-se a pessoas originárias da África, mas t
ambém pode ser usado d

In [ ]:
texto = "Considerando a palavra alvo \"negro\", identifique outras palavras no contexto que são semanticamente similares."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: SIMEÃO —O CRIOULO

AS VICTIMAS-ALGOZES

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
Jo
sé Mindlin

— 330 — 
assassino: amanhã eu lhe darei carta de liber­
dade. 
E accrescentou sem hesitar: 
— Vão prender Es
meria, a cúmplice de Pae-
Rayol ...
>>PERGUNTA<< Considerando a palavra alvo "negro", identifique outras palavras no con
texto que são semanticamente similares.
>>RESPOSTA<< A palavra "crioulo" é uma das palavras similares ao contexto, pois 
ela também se refere a um ser humano de cor preta ou afro-americano. Outras palavras similares incluem "preto", "afro", 
"black", entre outras.

Documentos de origem:
page_content='SIMEÃO —O CRIOULO' metadata={'page': 18, 'source': '/content/arquivo1.pdf', 'start_index': 0}
page_content='AS VICTIMAS-ALGOZES' metadata={'page': 2, 'source': '/content/arquivo1.pdf', 'start_index': 0}
p

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra \'negro\'"

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: — 25 — 
se explicam pela natureza da raça, o que seria 
absurdo; explicam-se pela condição de escravo, 
que 
corrompe e perverte o homem. 
O creoulo amorosamente creado pela familia 
dos senhores seria talvez o seu melhor amigo, 

se não fosse escravo.

- 67 — 
ideiado o corpo da esposa ou da filha de seo 
senhor. 
A escravidão é serpente: sua ling
oa derrama 
sempre veneno.

XIII 
O Senhor que se degrada ao ponto de dis­
tinguir como mulher uma sua escrava é mais 
d
o que immoral, é um imprudente e desassi-
sado que põe em desgoverno a própria caza, 
e levanta em throm do ignomínia a 
escra­
vidão corrupta elevada á senhora. 
Ha em semelhante erro lamentável esque­
cimento do dever, e sacrifício' de dig
nidade. 
A torpeza da escravidão é contagiosa e se ino-
cula na vida domestica do senhor que ouza 
expor-se ao contacto 
vergonhoso com a escra

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: — 25 — 
se explicam pela natureza da raça, o que seria 
absurdo; explicam-se pela condição de escravo, 
que 
corrompe e perverte o homem. 
O creoulo amorosamente creado pela familia 
dos senhores seria talvez o seu melhor amigo, 

se não fosse escravo.

- 67 — 
ideiado o corpo da esposa ou da filha de seo 
senhor. 
A escravidão é serpente: sua ling
oa derrama 
sempre veneno.

— 49 — 
O escravo é necessariamente máo e inimigo 
e seu senhor. 
A m"adre-féra escravidão f
az perversos, e vos 
arca de inimigos.

XIII 
O Senhor que se degrada ao ponto de dis­
tinguir como mulher uma sua escra
va é mais 
do que immoral, é um imprudente e desassi-
sado que põe em desgoverno a própria caza, 
e levanta em throm do 
ignomínia a escra­
vidão corrupta elevada á senhora. 
Ha em semelhante erro lamentável esque­
cimento do dever, e sacrif
ício' de dignidade. 
A

In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: — 25 — 
se explicam pela natureza da raça, o que seria 
absurdo; explicam-se pela condição de escravo, 
que 
corrompe e perverte o homem. 
O creoulo amorosamente creado pela familia 
dos senhores seria talvez o seu melhor amigo, 

se não fosse escravo.

- 67 — 
ideiado o corpo da esposa ou da filha de seo 
senhor. 
A escravidão é serpente: sua ling
oa derrama 
sempre veneno.

XIII 
O Senhor que se degrada ao ponto de dis­
tinguir como mulher uma sua escrava é mais 
d
o que immoral, é um imprudente e desassi-
sado que põe em desgoverno a própria caza, 
e levanta em throm do ignomínia a 
escra­
vidão corrupta elevada á senhora. 
Ha em semelhante erro lamentável esque­
cimento do dever, e sacrifício' de dig
nidade. 
A torpeza da escravidão é contagiosa e se ino-
cula na vida domestica do senhor que ouza 
expor-se ao contacto 
vergonhoso com a escra

In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: — 25 — 
se explicam pela natureza da raça, o que seria 
absurdo; explicam-se pela condição de escravo, 
que 
corrompe e perverte o homem. 
O creoulo amorosamente creado pela familia 
dos senhores seria talvez o seu melhor amigo, 

se não fosse escravo.

- 67 — 
ideiado o corpo da esposa ou da filha de seo 
senhor. 
A escravidão é serpente: sua ling
oa derrama 
sempre veneno.

— 160 — 
caria e a policia dorme sem jamais sonhar com 
essa entidade malvada. 
Mas em muita
s fazendas ha dissimulado, si­
nistro, fatal o negro feiticeiro. 
E o negro feiticeiro é um perigo real de todos 
os dia
s. 
Os outros escravos, se o conhecem, o temem, 
procurão torna-lo amigo com bajulações, pre-
zentes, serviços e obrigad
a submissão: se o 
não conhecem, sentem-no em males que expe-
rimentão. 
Em regra que poucas excepções concede, o 
negro
 escravo acredita no p

In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado_llm = avaliarContexto(texto)

# Mostra o resultado
print("Resposta:")
print_linhas_menores(resultado_llm["result"])

# Verifique os documentos de origem usados
print("\nDocumentos de origem:")
for fonte in resultado_llm["source_documents"]:
  print(fonte)

Resposta:
Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido
.
Contexto: — 25 — 
se explicam pela natureza da raça, o que seria 
absurdo; explicam-se pela condição de escravo, 
que 
corrompe e perverte o homem. 
O creoulo amorosamente creado pela familia 
dos senhores seria talvez o seu melhor amigo, 

se não fosse escravo.

XIII 
O Senhor que se degrada ao ponto de dis­
tinguir como mulher uma sua escrava é mais 
do qu
e immoral, é um imprudente e desassi-
sado que põe em desgoverno a própria caza, 
e levanta em throm do ignomínia a escr
a­
vidão corrupta elevada á senhora. 
Ha em semelhante erro lamentável esque­
cimento do dever, e sacrifício' de dignida
de. 
A torpeza da escravidão é contagiosa e se ino-
cula na vida domestica do senhor que ouza 
expor-se ao contacto verg
onhoso com a escrava. 
Acaba a disciplina e a ordem na caza: as 
outras escravas murmurão invejosas; a que foi 
distingu
ida levanta os olhos a

Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:07:12 (h:mm:ss)
